<h1> <i>Consejo Constitucional</i>  2023 Chile Elections   </h1>
<h2>  Combined votes and participation data to simplify data loading on looker studio  </h2>

In previous notebooks, we analyzed voting results by political sector, participation, and voter characterization data. We also added latitude and longitude coordinates to each voting location to facilitate geospatial data analysis.

When loading the data into Looker Studio, we encountered necessary data grouping requirements, which slowed down the chart refresh process.

In this section, we’ll create a new dataset that combines voting results by political pact, tracks the number of official votes, and includes participation data for each polling location. This will streamline the process in Looker Studio

In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

In [4]:
cc_data=pd.read_csv(r"C:\Users\gzuni\cc_final.csv")

C:\Users\gzuni\AppData\Local\Temp\ipykernel_9976\1027899099.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  cc_data=pd.read_csv(r"C:\Users\gzuni\cc_final.csv")


In [6]:
#We fill the NAN values for null and blanc votes for the fields 'Pacto', 'Lista', and 'Partido'
cc_data.loc[cc_data['Nombres'] == 'VOTOS NULOS', 'Pacto'] = 'NULOS'
cc_data.loc[cc_data['Nombres'] == 'VOTOS NULOS', 'Lista'] = 'NULOS'
cc_data.loc[cc_data['Nombres'] == 'VOTOS NULOS', 'Partido'] = 'NULOS'

cc_data.loc[cc_data['Nombres'] == 'VOTOS EN BLANCO', 'Pacto'] = 'BLANCOS'
cc_data.loc[cc_data['Nombres'] == 'VOTOS EN BLANCO', 'Lista'] = 'BLANCOS'
cc_data.loc[cc_data['Nombres'] == 'VOTOS EN BLANCO', 'Partido'] = 'BLANCOS'


In [8]:
cc_data.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,Lista,Pacto,Partido,Nombres,Primer apellido,Segundo apellido,cargo,votos,nombre_local_comuna,Latitude,Longitude,coordinates
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,A,PARTIDO DE LA GENTE,PARTIDO DE LA GENTE,CARLOS ALBERTO,MELLA,LATORRE,candidat@,38,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",-20.270390,-70.10254,"-20.27039,-70.10253999999999"
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,A,PARTIDO DE LA GENTE,PARTIDO DE LA GENTE,CLAUDIA EVELYN,VILLEGAS,CARVAJAL,candidat@,99,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",-20.239432,-70.05358,"-20.2394325,-70.0535802"


In [10]:
#We drop irrelevant data
cc_data_sum=cc_data.drop(columns=['Nombres','Primer apellido','Segundo apellido','cargo','Latitude','Longitude','coordinates'])


In [12]:
cc_data_sum.head(2)

#In chile political sectors reach agreements for the elections which are inscribed as 'Lists'. We'll take a look at the list at this particular election to classify votes
cc_data_sum['Lista'].unique()

array(['A', 'B', 'C', 'D', 'E', 'BLANCOS', 'NULOS', 'ZZI'], dtype=object)

In [14]:
grouped = cc_data_sum.groupby(['nro','Region','Circunscripcion senatorial','Distrito','Comuna','Circunscripcion electoral','Local','Lista','Pacto','Partido','nombre_local_comuna'], as_index=False)


cc_data_sum = grouped[['votos']].sum()



In [26]:
#Classifyin votes by belongin to offiacialdom or opposition
cc_data_sum.loc[cc_data['Lista'] == 'A', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data['Lista'] == 'B', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data['Lista'] == 'C', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data['Lista'] == 'D', 'bloque'] = 'Oficialismo'
cc_data_sum.loc[cc_data['Lista'] == 'E', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data['Lista'] =='ZZI', 'bloque'] = 'Independientes'
cc_data_sum.loc[cc_data['Lista'] == 'NULOS', 'bloque'] = 'NULOS'
cc_data_sum.loc[cc_data['Lista'] == 'BLANCOS', 'bloque'] = 'BLANCOS'

In [28]:
cc_data_sum.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,Lista,Pacto,Partido,nombre_local_comuna,votos,bloque
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,A,PARTIDO DE LA GENTE,PARTIDO DE LA GENTE,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",231,oposicion
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,B,TODO POR CHILE,PARTIDO DEMOCRATA CRISTIANO,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",109,oposicion


In [30]:
len(cc_data_sum)

46618

In [32]:
#we extract the results of percentage of votes obtain by officialism
percent_of=pd.read_excel(r"C:\Users\gzuni\cc_por_local - copia.xlsx")

In [34]:
percent_of.columns

Index(['nro', 'Region', 'Circunscripcion senatorial', 'Distrito', 'Comuna',
       'Circunscripcion electoral', 'Local', 'nombre_local_comuna', 'votos',
       'ACCION HUMANISTA', 'BLANCOS', 'COMUNES', 'CONVERGENCIA SOCIAL',
       'EVOLUCION POLITICA', 'FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE', 'INDEPENDIENTE ACCION HUMANISTA',
       'INDEPENDIENTE CONVERGENCIA SOCIAL', 'INDEPENDIENTE EVOLUCION POLITICA',
       'INDEPENDIENTE FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE PARTIDO COMUNISTA DE CHILE',
       'INDEPENDIENTE PARTIDO DEMOCRATA CRISTIANO',
       'INDEPENDIENTE PARTIDO LIBERAL DE CHILE',
       'INDEPENDIENTE PARTIDO POR LA DEMOCRACIA',
       'INDEPENDIENTE PARTIDO RADICAL DE CHILE',
       'INDEPENDIENTE PARTIDO SOCIALISTA DE CHILE',
       'INDEPENDIENTE RENOVACION NACIONAL',
       'INDEPENDIENTE REVOLUCION DEMOCRATICA',
       'INDEPENDIENTE UNION DEMOCRATA INDEPENDIENTE', 'NULOS',
       'PARTIDO COMUNISTA DE CHILE', 'PARTIDO DE L

In [36]:
oficialismo=percent_of.loc[:,['nombre_local_comuna','Oficialismo','oposicion','votos','Inscritos','% participacion','total_votantes']]

In [38]:
oficialismo['%oficialismo']=oficialismo['Oficialismo']/oficialismo['votos']
oficialismo['%oposicion']=oficialismo['oposicion']/oficialismo['votos']


In [40]:
oficialismo

,nombre_local_comuna,Oficialismo,oposicion,votos,Inscritos,% participacion,total_votantes,%oficialismo,%oposicion
0,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",274,1544,2525,3264.0,0.773591,2520.0,0.108515,0.611485
1,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",7,23,38,56.0,0.678571,38.0,0.184211,0.605263
2,"COLEGIO CATOLICO NAZARET, ALTO HOSPICIO",565,3067,5069,6400.0,0.792031,5038.0,0.111462,0.605050
3,"COLEGIO MARISTA HERMANO FERNANDO, ALTO HOSPICIO",495,2589,4331,5600.0,0.773393,4329.0,0.114292,0.597783
4,"COLEGIO METODISTA WILLIAM TAYLOR, ALTO HOSPICIO",556,2765,4749,6000.0,0.791500,4763.0,0.117077,0.582228
...,...,...,...,...,...,...,...,...,...
2927,"LICEO BICENTENARIO DE TREHUACO, TREHUACO",203,889,1387,1535.0,0.903583,1390.0,0.146359,0.640952
2928,"ESCUELA FERNANDO BAQUEDANO, YUNGAY",945,3791,6174,6800.0,0.907941,6178.0,0.153061,0.614027
2929,"LICEO CAMPANARIO, YUNGAY",348,1389,2378,2646.0,0.898715,2379.0,0.146341,0.584104
2930,"LICEO DE YUNGAY, YUNGAY",808,3868,6245,7322.0,0.852909,6254.0,0.129384,0.619376


In [42]:
cc_data_sum['%oficialismo']=0
cc_data_sum['%oposicion']=0
cc_data_sum['total_votantes']=0
cc_data_sum['Inscritos']=0
cc_data_sum['% participacion']=0

In [44]:
cc_data_sum.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,Lista,Pacto,Partido,nombre_local_comuna,votos,bloque,%oficialismo,%oposicion,total_votantes,Inscritos,% participacion
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,A,PARTIDO DE LA GENTE,PARTIDO DE LA GENTE,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",231,oposicion,0,0,0,0,0
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,B,TODO POR CHILE,PARTIDO DEMOCRATA CRISTIANO,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",109,oposicion,0,0,0,0,0


In [46]:
#We add the totalized results of participation , %officialdom and total voters by polling location only one time to the data containing total votes by political sector

merged_df = cc_data_sum.merge(
    oficialismo[['nombre_local_comuna', '%oficialismo', '%oposicion','total_votantes','Inscritos','% participacion']], 
    on='nombre_local_comuna', 
    how='left', 
    suffixes=('', '_oficialismo')
)

# Create a mask for 'UNIDAD PARA CHILE'
mask = (cc_data_sum['Pacto'] == 'UNIDAD PARA CHILE')

# Iterate through unique 'nombre_local_comuna'
for local in cc_data_sum.loc[mask, 'nombre_local_comuna'].unique():
    # Find the first occurrence where 'Pacto' is 'UNIDAD PARA CHILE'
    first_occurrence_idx = cc_data_sum[(cc_data_sum['nombre_local_comuna'] == local) & mask].index[0]
    # Update the '%oficialismo' and '%oposición' columns
    cc_data_sum.at[first_occurrence_idx, '%oficialismo'] = merged_df.at[first_occurrence_idx, '%oficialismo_oficialismo']
    cc_data_sum.at[first_occurrence_idx, '%oposicion'] = merged_df.at[first_occurrence_idx, '%oposicion_oficialismo']
    cc_data_sum.at[first_occurrence_idx, 'total_votantes'] = merged_df.at[first_occurrence_idx, 'total_votantes_oficialismo']
    cc_data_sum.at[first_occurrence_idx, 'Inscritos'] = merged_df.at[first_occurrence_idx, 'Inscritos_oficialismo']
    cc_data_sum.at[first_occurrence_idx, '% participacion'] = merged_df.at[first_occurrence_idx, '% participacion_oficialismo']

In [50]:

cc_data_sum.loc[cc_data_sum['Lista'] == 'D', 'bloque'] = 'Oficialismo'
cc_data_sum.loc[cc_data_sum['Lista'] == 'A', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data_sum['Lista'] == 'B', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data_sum['Lista'] == 'C', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data_sum['Lista'] == 'E', 'bloque'] = 'oposicion'
cc_data_sum.loc[cc_data_sum['Lista'] == 'ZZI', 'bloque'] = 'independientes'
cc_data_sum.loc[cc_data_sum['Lista'] == 'NULOS', 'bloque'] = 'nulos'
cc_data_sum.loc[cc_data_sum['Lista'] == 'BLANCOS', 'bloque'] = 'blancos'

In [56]:
cc_data_sum.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,Lista,Pacto,Partido,nombre_local_comuna,votos,bloque,%oficialismo,%oposicion,total_votantes,Inscritos,% participacion
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,A,PARTIDO DE LA GENTE,PARTIDO DE LA GENTE,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",231,oposicion,0.0,0.0,0.0,0.0,0.0
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,B,TODO POR CHILE,PARTIDO DEMOCRATA CRISTIANO,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",109,oposicion,0.0,0.0,0.0,0.0,0.0


In [54]:
#Exporting the data to be used in looker studio
cc_data_sum.to_excel('cc2023.xlsx',index=False)
cc_data_sum.to_csv('cc2023.csv',index=False)

Calculating the totals per polling location on each field available in SERVEL data


In [123]:
# Calculate the total votes for each 'nombre_local_comuna'
total_votes = cc_data_sum.groupby('nombre_local_comuna')['votos'].sum().reset_index()
total_votes = total_votes.rename(columns={'votos': 'total_votos'})




In [124]:
total_votes.head(2)

,nombre_local_comuna,total_votos
0,"ALIANZA FRANCESA, VITACURA",4980
1,"ALIANZA FRANCESA, VIÑA DEL MAR",7835


In [142]:
total_partido = cc_data_sum.groupby(['nombre_local_comuna','Partido'])['votos'].sum().reset_index()
total_partido = total_partido.rename(columns={'votos': 'total_partido'})

In [143]:
total_partido.head(2)

,nombre_local_comuna,Partido,total_partido
0,"ALIANZA FRANCESA, VITACURA",BLANCOS,70
1,"ALIANZA FRANCESA, VITACURA",COMUNES,36


In [144]:
total_partido = total_partido.pivot_table(
    index='nombre_local_comuna',
    columns=['Partido'],
    values='total_partido',
    aggfunc='sum',
    fill_value=0
)


In [145]:
total_partido=total_partido.reset_index()
total_partido.head(2)

Partido,nombre_local_comuna,ACCION HUMANISTA,BLANCOS,COMUNES,CONVERGENCIA SOCIAL,EVOLUCION POLITICA,FEDERACION REGIONALISTA VERDE SOCIAL,INDEPENDIENTE,INDEPENDIENTE ACCION HUMANISTA,INDEPENDIENTE CONVERGENCIA SOCIAL,...,PARTIDO DE LA GENTE,PARTIDO DEMOCRATA CRISTIANO,PARTIDO LIBERAL DE CHILE,PARTIDO POR LA DEMOCRACIA,PARTIDO RADICAL DE CHILE,PARTIDO REPUBLICANO DE CHILE,PARTIDO SOCIALISTA DE CHILE,RENOVACION NACIONAL,REVOLUCION DEMOCRATICA,UNION DEMOCRATA INDEPENDIENTE
0,"ALIANZA FRANCESA, VITACURA",0,70,36,180,1061,0,0,0,0,...,27,107,15,171,17,1876,31,135,35,665
1,"ALIANZA FRANCESA, VIÑA DEL MAR",0,188,16,448,0,0,0,0,0,...,0,75,0,138,30,3247,176,167,212,835


In [147]:
total_pacto = cc_data_sum.groupby(['nombre_local_comuna','Pacto'])['votos'].sum().reset_index()
total_pacto = total_pacto.rename(columns={'votos': 'total_pacto'})
total_pacto = total_pacto.pivot_table(
    index='nombre_local_comuna',
    columns=['Pacto'],
    values='total_pacto',
    aggfunc='sum',
    fill_value=0
)
total_pacto.drop(columns=['BLANCOS','NULOS'], inplace=True)
total_pacto=total_pacto.reset_index()
total_pacto.head(2)

Pacto,nombre_local_comuna,CHILE SEGURO,INDEPENDIENTES,PARTIDO DE LA GENTE,PARTIDO REPUBLICANO DE CHILE,TODO POR CHILE,UNIDAD PARA CHILE
0,"ALIANZA FRANCESA, VITACURA",2229,0,27,1876,295,342
1,"ALIANZA FRANCESA, VIÑA DEL MAR",2045,0,0,3247,339,1398


In [148]:
total_bloque = cc_data_sum.groupby(['nombre_local_comuna','bloque'])['votos'].sum().reset_index()
total_bloque = total_bloque.rename(columns={'votos': 'total_bloque'})
total_bloque = total_bloque.pivot_table(
    index='nombre_local_comuna',
    columns=['bloque'],
    values='total_bloque',
    aggfunc='sum',
    fill_value=0)

total_bloque.drop(columns=['BLANCOS','NULOS'], inplace=True)
total_bloque=total_bloque.reset_index()
total_bloque.head(2)

bloque,nombre_local_comuna,Oficialismo,oposicion
0,"ALIANZA FRANCESA, VITACURA",342,4427
1,"ALIANZA FRANCESA, VIÑA DEL MAR",1398,5631


In [149]:
test=total_bloque[(total_bloque['nombre_local_comuna']=='UNIVERSIDAD DE MAGALLANES, PUNTA ARENAS')]
test

bloque,nombre_local_comuna,Oficialismo,oposicion
2924,"UNIVERSIDAD DE MAGALLANES, PUNTA ARENAS",737,2437


In [150]:

grouped_2 = cc_data_sum.groupby(['nro','Region','Circunscripcion senatorial','Distrito','Comuna','Circunscripcion electoral','Local','nombre_local_comuna'], as_index=False)


cc_data_sum=grouped_2[['votos']].sum()

cc_data_sum.head(10)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,nombre_local_comuna,votos
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",2525
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,C.P. ALTO HOSPICIO,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",38
2,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO CATOLICO NAZARET,"COLEGIO CATOLICO NAZARET, ALTO HOSPICIO",5069
3,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO MARISTA HERMANO FERNANDO,"COLEGIO MARISTA HERMANO FERNANDO, ALTO HOSPICIO",4331
4,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO METODISTA WILLIAM TAYLOR,"COLEGIO METODISTA WILLIAM TAYLOR, ALTO HOSPICIO",4749
5,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO SAGRADO CORAZON DE JESUS,"COLEGIO SAGRADO CORAZON DE JESUS, ALTO HOSPICIO",4987
6,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO SALESIANO SANTO DOMINGO SAVIO,"COLEGIO SALESIANO SANTO DOMINGO SAVIO, ALTO HO...",4916
7,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO SAN ANTONIO DE MATILLA,"COLEGIO SAN ANTONIO DE MATILLA, ALTO HOSPICIO",5100
8,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,COLEGIO SIMON BOLIVAR,"COLEGIO SIMON BOLIVAR, ALTO HOSPICIO",5058
9,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,LICEO BICENTENARIO COLEGIO NIRVANA,"LICEO BICENTENARIO COLEGIO NIRVANA, ALTO HOSPICIO",5067


In [151]:
len(cc_data_sum)

2932

In [152]:
cc_data_merge= cc_data_sum.merge(total_partido, on='nombre_local_comuna', how='left')

In [153]:
cc_data_merge.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,nombre_local_comuna,votos,ACCION HUMANISTA,...,PARTIDO DE LA GENTE,PARTIDO DEMOCRATA CRISTIANO,PARTIDO LIBERAL DE CHILE,PARTIDO POR LA DEMOCRACIA,PARTIDO RADICAL DE CHILE,PARTIDO REPUBLICANO DE CHILE,PARTIDO SOCIALISTA DE CHILE,RENOVACION NACIONAL,REVOLUCION DEMOCRATICA,UNION DEMOCRATA INDEPENDIENTE
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",2525,42,...,231,109,0,127,11,766,39,178,0,122
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,C.P. ALTO HOSPICIO,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",38,0,...,8,3,0,0,0,9,5,1,0,2


In [154]:
total_pacto=total_pacto.rename(columns={'PARTIDO REPUBLICANO DE CHILE':'partido republicano_pacto','PARTIDO DE LA GENTE':'partido de la gente_pacto'})
cc_data_merge= cc_data_merge.merge(total_pacto, on='nombre_local_comuna', how='left')

In [155]:
cc_data_merge.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,nombre_local_comuna,votos,ACCION HUMANISTA,...,PARTIDO SOCIALISTA DE CHILE,RENOVACION NACIONAL,REVOLUCION DEMOCRATICA,UNION DEMOCRATA INDEPENDIENTE,CHILE SEGURO,INDEPENDIENTES,partido de la gente_pacto,partido republicano_pacto,TODO POR CHILE,UNIDAD PARA CHILE
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",2525,42,...,39,178,0,122,300,0,231,766,247,274
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,C.P. ALTO HOSPICIO,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",38,0,...,5,1,0,2,3,0,8,9,3,7


In [156]:
cc_data_merge= cc_data_merge.merge(total_bloque, on='nombre_local_comuna', how='left')

In [157]:
cc_data_merge.head(2)

,nro,Region,Circunscripcion senatorial,Distrito,Comuna,Circunscripcion electoral,Local,nombre_local_comuna,votos,ACCION HUMANISTA,...,REVOLUCION DEMOCRATICA,UNION DEMOCRATA INDEPENDIENTE,CHILE SEGURO,INDEPENDIENTES,partido de la gente_pacto,partido republicano_pacto,TODO POR CHILE,UNIDAD PARA CHILE,Oficialismo,oposicion
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",2525,42,...,0,122,300,0,231,766,247,274,274,1544
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,C.P. ALTO HOSPICIO,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",38,0,...,0,2,3,0,8,9,3,7,7,23


In [158]:
cc_data_merge.to_excel('cc_pivot.xlsx',index=False)

In [159]:
len(cc_data_merge)

2932

In [160]:
participacion=pd.read_csv(r"C:\Users\gzuni\participation_final.csv")

In [161]:
participacion.head(2)

,Nro.Región,Región,Circunscripción senatorial,Distrito,Comuna,Circunscripción electoral,Local,Votos,Inscritos,nombre_local_comuna,% participacion,no_votan,Latitude,Longitude,coordinates
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,2525,3264,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",0.773591,739,-20.270390,-70.10254,"-20.27039,-70.10253999999999"
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,C.P. ALTO HOSPICIO,38,56,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",0.678571,18,-20.239432,-70.05358,"-20.2394325,-70.0535802"


In [162]:
len(participacion)

2932

In [163]:
participacion=participacion.drop(columns=['Nro.Región','Región','Circunscripción senatorial','Distrito','Comuna','Circunscripción electoral','Local','Votos','Latitude','Longitude'])


In [165]:
participacion.head(2)

,Inscritos,nombre_local_comuna,% participacion,no_votan,coordinates
0,3264,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",0.773591,739,"-20.27039,-70.10253999999999"
1,56,"C.P. ALTO HOSPICIO, ALTO HOSPICIO",0.678571,18,"-20.2394325,-70.0535802"


In [166]:
cc_data_merge= cc_data_merge.merge(participacion, on='nombre_local_comuna', how='left')
cc_data_merge.columns

Index(['nro', 'Region', 'Circunscripcion senatorial', 'Distrito', 'Comuna',
       'Circunscripcion electoral', 'Local', 'nombre_local_comuna', 'votos',
       'ACCION HUMANISTA', 'BLANCOS', 'COMUNES', 'CONVERGENCIA SOCIAL',
       'EVOLUCION POLITICA', 'FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE', 'INDEPENDIENTE ACCION HUMANISTA',
       'INDEPENDIENTE CONVERGENCIA SOCIAL', 'INDEPENDIENTE EVOLUCION POLITICA',
       'INDEPENDIENTE FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE PARTIDO COMUNISTA DE CHILE',
       'INDEPENDIENTE PARTIDO DEMOCRATA CRISTIANO',
       'INDEPENDIENTE PARTIDO LIBERAL DE CHILE',
       'INDEPENDIENTE PARTIDO POR LA DEMOCRACIA',
       'INDEPENDIENTE PARTIDO RADICAL DE CHILE',
       'INDEPENDIENTE PARTIDO SOCIALISTA DE CHILE',
       'INDEPENDIENTE RENOVACION NACIONAL',
       'INDEPENDIENTE REVOLUCION DEMOCRATICA',
       'INDEPENDIENTE UNION DEMOCRATA INDEPENDIENTE', 'NULOS',
       'PARTIDO COMUNISTA DE CHILE', 'PARTIDO DE L

In [167]:
descr_votantes=pd.read_csv(r"C:\Users\gzuni\descr_votantes_final.csv")
descr_votantes.head(2)

,Nro.Región,Región,Circunscripción senatorial,Distrito,Comuna,Circunscripción electoral,Local,Sexo,Rango etario,Nacionalidad,Votantes,nombre_local_comuna,Total_Inscritos,Latitude,Longitude,coordinates,Total_Votantes
0,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,HOMBRES,18-19,BOLIVIA,3,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",3264,-20.27039,-70.10254,"-20.27039,-70.10253999999999",2520
1,1,DE TARAPACA,CIRCUNSCRIPCION SENATORIAL 2,DISTRITO 2,ALTO HOSPICIO,ALTO HOSPICIO,ANEXO DE COLEGIO SAN ANTONIO DE MATILLA,HOMBRES,18-19,CHILE,68,"ANEXO DE COLEGIO SAN ANTONIO DE MATILLA, ALTO ...",3264,-20.27039,-70.10254,"-20.27039,-70.10253999999999",2520


In [168]:

total_rango = descr_votantes.groupby(['nombre_local_comuna','Rango etario'])['Votantes'].sum().reset_index()
total_rango = total_rango.rename(columns={'Votantes': 'total_rango'})
total_rango = total_rango.pivot_table(
    index='nombre_local_comuna',
    columns=['Rango etario'],
    values='total_rango',
    aggfunc='sum',
    fill_value=0)

total_rango=total_rango.reset_index()

total_rango.head(2)

Rango etario,nombre_local_comuna,18-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
0,"ALIANZA FRANCESA, VITACURA",92,666,821,846,830,728,532,475
1,"ALIANZA FRANCESA, VIÑA DEL MAR",249,1296,1500,1239,1161,1407,730,239


In [169]:
len(total_rango)

2932

In [170]:
total_sexo = descr_votantes.groupby(['nombre_local_comuna','Sexo'])['Votantes'].sum().reset_index()
total_sexo = total_sexo.rename(columns={'Votantes': 'total_sexo'})
total_sexo = total_sexo.pivot_table(
    index='nombre_local_comuna',
    columns=['Sexo'],
    values='total_sexo',
    aggfunc='sum',
    fill_value=0)

total_sexo=total_sexo.reset_index()

total_sexo.head(2)

Sexo,nombre_local_comuna,HOMBRES,MUJERES
0,"ALIANZA FRANCESA, VITACURA",2153,2837
1,"ALIANZA FRANCESA, VIÑA DEL MAR",3726,4095


In [171]:
len(total_sexo)

2932

In [173]:
total_sexo['total_votantes']=total_sexo['HOMBRES']+total_sexo['MUJERES']
total_sexo.head(2)

Sexo,nombre_local_comuna,HOMBRES,MUJERES,total_votantes
0,"ALIANZA FRANCESA, VITACURA",2153,2837,4990
1,"ALIANZA FRANCESA, VIÑA DEL MAR",3726,4095,7821


In [174]:
cc_data_merge= cc_data_merge.merge(total_rango, on='nombre_local_comuna', how='left')
cc_data_merge= cc_data_merge.merge(total_sexo, on='nombre_local_comuna', how='left')

cc_data_merge.columns

Index(['nro', 'Region', 'Circunscripcion senatorial', 'Distrito', 'Comuna',
       'Circunscripcion electoral', 'Local', 'nombre_local_comuna', 'votos',
       'ACCION HUMANISTA', 'BLANCOS', 'COMUNES', 'CONVERGENCIA SOCIAL',
       'EVOLUCION POLITICA', 'FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE', 'INDEPENDIENTE ACCION HUMANISTA',
       'INDEPENDIENTE CONVERGENCIA SOCIAL', 'INDEPENDIENTE EVOLUCION POLITICA',
       'INDEPENDIENTE FEDERACION REGIONALISTA VERDE SOCIAL',
       'INDEPENDIENTE PARTIDO COMUNISTA DE CHILE',
       'INDEPENDIENTE PARTIDO DEMOCRATA CRISTIANO',
       'INDEPENDIENTE PARTIDO LIBERAL DE CHILE',
       'INDEPENDIENTE PARTIDO POR LA DEMOCRACIA',
       'INDEPENDIENTE PARTIDO RADICAL DE CHILE',
       'INDEPENDIENTE PARTIDO SOCIALISTA DE CHILE',
       'INDEPENDIENTE RENOVACION NACIONAL',
       'INDEPENDIENTE REVOLUCION DEMOCRATICA',
       'INDEPENDIENTE UNION DEMOCRATA INDEPENDIENTE', 'NULOS',
       'PARTIDO COMUNISTA DE CHILE', 'PARTIDO DE L

In [175]:
cc_data_merge.to_excel('cc_por_local.xlsx', index=False)

In [176]:
len(cc_data_merge)

2932